In [1]:
import numpy as np
import scipy.linalg as la
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import geomstats as gs
import geomstats.geometry.spd_matrices as spd
import pickle
from scipy.signal import butter, lfilter
from geomstats.learning.frechet_mean import FrechetMean
from scipy.io import loadmat
import pandas as pd
from copy import deepcopy
from scipy.stats import wilcoxon


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.3f}".format

INFO: Using numpy backend


In [2]:
def getCovariances(X):
    covlist = np.empty((X.shape[0], X.shape[2], X.shape[2]))
    for i in range(len(X)):
        trial = X[i]
        n = trial.shape[0]
        epoch = (trial - np.mean(trial))/np.std(trial)
        covlist[i] = np.dot(trial.T, trial)/n
    return covlist

def bandpass(X, fs, fl, fh, order):
    X_filtered = np.empty_like(X)    
    for i in range(X.shape[1]):
        trial = X[:, i, :]
        nyq = 0.5 * fs
        low = fl / nyq
        high = fh / nyq
        b, a = butter(order, [low, high], btype='band')
        trial = lfilter(b, a, trial, axis=0)
        X_filtered[:, i, :] = trial
            
    return X_filtered

def getData(session, fs=1000, fl=8, fh=30, order = 5):
    channels = [7, 8, 9, 10, 12, 13, 14, 17, 18, 19, 20, 32, 33, 34, 35, 36, 37, 38, 39, 40]
    SubjectsCOV = []
    SubjectsY = []
    for i in range(1, 55):
        filename = 'datasets/54subjects/'+ session + '/subj{}_EEG_MI.mat'.format(i)
        data = loadmat(filename)
        
        d_train = data['EEG_MI_train']
        Y_train = d_train['y_dec'][0][0][0] - 1
        
        d_test = data['EEG_MI_test']
        Y_test = d_test['y_dec'][0][0][0] - 1
        
        X_train = d_train['smt'][0][0][:,:,channels]
        X_train_filtered = bandpass(X_train, fs, fl, fh, order)
        X_train_filtered = X_train_filtered[1000:3500]
        
        X_test = d_test['smt'][0][0][:,:,channels]
        X_test_filtered = bandpass(X_test, fs, fl, fh, order)
        X_test_filtered = X_test_filtered[1000:3500]
        
        X = np.concatenate((X_train_filtered, X_test_filtered), axis = 1)
        X = np.swapaxes(X, 0, 1)
        covX = getCovariances(X)
        Y = np.concatenate((Y_train, Y_test))    
        SubjectsCOV.append(covX)
        SubjectsY.append(Y)
                    
    return SubjectsCOV, SubjectsY

In [3]:
SubjectsCOV, SubjectsY = getData("sess01")

In [4]:
filename = 'datasets/54COV7Sess01.pickle'
outfile = open(filename,'wb')
pickle.dump([SubjectsCOV, SubjectsY], outfile)
outfile.close()

In [5]:
SubjectsCOV, SubjectsY = getData("sess02")

In [6]:
filename = 'datasets/54COV7Sess02.pickle'
outfile = open(filename,'wb')
pickle.dump([SubjectsCOV, SubjectsY], outfile)
outfile.close()